# Python code version with pulp optimizer

In [2]:
from pulp import *
import pandas as pd

In [3]:
# import climate data. Here I choose 3 of them
hw85 = pd.read_csv( r"C:\\Users\\NLLIST\\OneDrive - Sweco AB\\Thesis\\Climate data\\rcp85\\hotdays.csv", delimiter=',' , header=None)
hr85 = pd.read_csv( r"C:\\Users\\NLLIST\\OneDrive - Sweco AB\\Thesis\\Climate data\\rcp85\\mediumrain.csv", delimiter=',' , header=None)
sp85=  pd.read_csv( r"C:\Users\NLLIST\OneDrive - Sweco AB\Thesis\Climate data\rcp85\sunpeak.csv", delimiter=' ' , header=None)

In [4]:
#import buildings data, derived from processing of 
pand = pd.read_csv(r"C:\Users\NLLIST\Desktop\Thesis\try6\Merged.csv")

In [5]:
#Roofs parameters, work with dict
a=pand["_area"] 
roofs = range(len(a)) 
a=dict(zip(roofs,pand['_area']))

#green
rg=dict(zip(roofs,pand['rg'])) 
ig=dict(zip(roofs,pand['ig']))

#blue
rb=dict(zip(roofs,pand['rb']))

#yellow
ry = dict(zip(roofs,pand['ry']))

In [6]:
#Roofs parameters, depending also on climate variables.
#blue

cb85=pand["cb"]/(1000*len(hr85)) 

r85=hr85[1]

#green

# we cosider heat can be mitigated thanks to the presence of green roofs only when the
# temperatures are between 25 (excluded) and 31 degrees. A higher temperature won't allow for any relief, 
# while a lower temprature is not considered as damaging

#so we scale impacted people amount ig on the temperature, assuming it is max in correspondence of 31 degrees

h85=hw85[1]

h85 = h85[(h85 >=25) & (h85 <=31)]
h85= h85.reset_index(drop=True) # indexing has to be updated in order to further process the vector

w, r = len(h85), len(a); #create a matrix with dimensions roofs x heat events
m85 = [[0 for j in range(w)] for i in range(r)] 
for i in range(r):
    for j in range(w):
        m85[i][j]=ig[i]+(ig[i]/5)*(h85[j]-30)        

m85=pd.Series(m85).fillna(0).tolist()

#yellow
ds85=int(sp85.sum())


In [7]:
#Model parameters. They are uncertain but here considered 
cg= 5000
kb=0.8
kg=0.8
Q=187.5
theta= 0.05
hg=10
cy=0.0700734/1000

# technical parameters not uncertain
hg=10 # DEPTH OF WATER RETENTION LAYER for green roof

# Choose the budget
B=5000000

# # Choose the subsidy rate 
sub=0.2

# # Prices per m2 of roof type
py=315 #YR
pb=30 #BR
pg=80 #GR

In [8]:
#Big M value for big M method used to linearize a non-linear expression
M=99999

In [9]:
# choose the solver that pulp will use. The one mentioned here comes with the pulp package
solver = PULP_CBC_CMD(keepFiles=True)

## Model is constructed here in its linearized version 

In [10]:
model = LpProblem("Roofs",LpMaximize)

yGR = LpVariable.dicts("Green_roof",roofs,cat='Binary')
yBR = LpVariable.dicts("Blue_roof",roofs,cat='Binary')
xYR = LpVariable.dicts("Yellow_roof",roofs,lowBound=0,cat='Continuous')

bGR = LpVariable.dict("Green_roof_benefit", range(1), lowBound=0,cat='Continuous')
bBR = LpVariable.dict("Blue_roof_benefit", range(1), lowBound=0,cat='Continuous')
bYR = LpVariable.dict("Yellow_roof_benefit",range(1), lowBound=0,cat='Continuous')

s1 = LpVariable.dicts("slack1",roofs,lowBound=0,cat='Continuous')
s2 = LpVariable.dicts("slack2",roofs,lowBound=0,cat='Continuous')


for i in roofs:
    model+= xYR[i] <= a[i]
    model+= yGR[i]>=yBR[i] 
    model+= s1[i]+s2[i]==xYR[i]
    model+= s1[i]<=M*yGR[i]
    model+= s2[i]<=M*(1-yGR[i])
    
model+= lpSum(sub*(pg*yGR[i]*a[i] + pb*yBR[i]*a[i] + py*xYR[i] ) for i in roofs) <= B 
model+= bGR== lpSum(lpSum((1/3)*cg*m85[i][w] *rg[i] *yGR[i] for w in range(len(h85))) + lpSum((1/3)*cg*m45[i][w] *rg[i] *yGR[i] for w in range(len(h45))) + lpSum((1/3)*cg*m26[i][w] *rg[i] *yGR[i] for w in range(len(h26))) for i in roofs)
model+= bBR==lpSum(lpSum((1/3)*cb85[i]*a[i]*(kb*r85[j]*yBR[i]+kg*hg*yGR[i])*rb[i] for j in range(len(r85))) + lpSum((1/3)*cb45[i]*a[i]*(kb*r45[j]*yBR[i]+kg*hg*yGR[i])*rb[i] for j in range(len(r45))) + lpSum((1/3)*cb26[i]*a[i]*(kb*r26[j]*yBR[i]+kg*hg*yGR[i])*rb[i] for j in range(len(r26))) for i in roofs)
model+= bYR==lpSum((1/3)*cy*Q*ds85*ry[i]*(xYR[i]+theta*s1[i]) + (1/3)*cy*Q*ds45*ry[i]*(xYR[i]+theta*s1[i]) + (1/3)*cy*Q*ds26*ry[i]*(xYR[i]+theta*s1[i]) for i in roofs)

model+= lpSum(bGR)+lpSum(bBR)+lpSum(bYR) 


In [11]:
model.solve(solver)

1

In [12]:
print("Status:", LpStatus[model.status])

Status: Optimal


In [13]:
for v in model.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

Blue_roof_667 = 1.0
Blue_roof_benefit_0 = 51591.11
Green_roof_1002 = 1.0
Green_roof_1044 = 1.0
Green_roof_1056 = 1.0
Green_roof_270 = 1.0
Green_roof_279 = 1.0
Green_roof_328 = 1.0
Green_roof_352 = 1.0
Green_roof_361 = 1.0
Green_roof_394 = 1.0
Green_roof_434 = 1.0
Green_roof_470 = 1.0
Green_roof_526 = 1.0
Green_roof_563 = 1.0
Green_roof_597 = 1.0
Green_roof_605 = 1.0
Green_roof_667 = 1.0
Green_roof_681 = 1.0
Green_roof_823 = 1.0
Green_roof_841 = 1.0
Green_roof_benefit_0 = 1002413.2
Yellow_roof_1002 = 1853.8746
Yellow_roof_1004 = 1061.351
Yellow_roof_1014 = 1062.6734
Yellow_roof_1017 = 1226.6301
Yellow_roof_1087 = 448.17837
Yellow_roof_1094 = 1531.8447
Yellow_roof_1103 = 530.98979
Yellow_roof_259 = 781.60276
Yellow_roof_262 = 4907.5979
Yellow_roof_270 = 510.86801
Yellow_roof_284 = 159.38031
Yellow_roof_290 = 703.29557
Yellow_roof_301 = 2390.7282
Yellow_roof_320 = 452.37141
Yellow_roof_352 = 1259.0086
Yellow_roof_394 = 1490.6802
Yellow_roof_396 = 865.60338
Yellow_roof_417 = 316.56426
Yell

In [30]:
ind=[]


gr_v=[]
br_v=[]
yr_v=[]
ro=len(a)

gr_ben=model.variables()[2*ro+1].varValue
br_ben=model.variables()[ro].varValue
yr_ben=model.variables()[3*ro+2].varValue
for i in range(ro):
    ind.append(str(model.variables()[i]).split('_')[2])
    
    br_v.append(model.variables()[i].varValue)
    gr_v.append(model.variables()[i+ro+1].varValue)
    yr_v.append(model.variables()[i+2*ro+2].varValue)

# In[169]:

ind2 = list(map(int, ind))
gr=gr_v
br=br_v
yr=yr_v

j=0
for i in ind2:
    gr[i]=gr_v[j]*a[i]
    br[i]=br_v[j]*a[i]
    yr[i]=yr_v[j]
    
    j=j+1
    


In [31]:
ind=pd.DataFrame(ind, columns=['ind'])
gr_v=pd.DataFrame(gr_v, columns=['gr'])
br_v=pd.DataFrame(br_v, columns=['br'])
yr_v=pd.DataFrame(yr_v, columns=['yr'])

In [ ]:
p=pd.concat([pand.iloc[:,0],gr,br,yr],axis=1  ) 
benefits=['heat mitigation benefits:', str(gr_ben), 'flooding mitigation benfits:', str(br_ben)]

In [ ]:
p.to_csv(r"pathout\filename.csv")

In [ ]:
textfile=open(pathout\file.txt, 'w')
for element in benefits:
    textfile.write(element +'\n')
textfile.close()